In [3]:
import torch
import torch.nn as nn
# inputs = torch.rand(64, 3, 244, 244)
# conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
# out = conv(inputs)

# # Example of using Sequential
#         model = nn.Sequential(
#                   nn.Conv2d(1,20,5),
#                   nn.ReLU(),
#                   nn.Conv2d(20,64,5),
#                   nn.ReLU()
#                 )

#         # Example of using Sequential with OrderedDict
#         model = nn.Sequential(OrderedDict([
#                   ('conv1', nn.Conv2d(1,20,5)),
#                   ('relu1', nn.ReLU()),
#                   ('conv2', nn.Conv2d(20,64,5)),
#                   ('relu2', nn.ReLU())
#                 ]))





IndentationError: unexpected indent (<ipython-input-3-98026b652870>, line 8)

In [17]:
#https://blog.csdn.net/dss_dssssd/article/details/82980222

# hyper parameters
in_dim=1
n_hidden_1=1
n_hidden_2=1
out_dim=1

class Net(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super().__init__()

        self.layer = nn.Sequential(
            nn.Linear(in_dim, n_hidden_1),
            nn.ReLU(True),
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(True),
            # 最后一层不需要添加激活函数
            nn.Linear(n_hidden_2, out_dim)
             )

    def forward(self, x):
      	x = self.layer(x)
      	return  print(x)
    

   

NameError: name 'self' is not defined

In [21]:
#nn与nn.function 区别
#1.nn
class CNN(nn.Module):
    
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1,  out_channels=16, kernel_size=5,padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5,  padding=0)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        self.linear1 = nn.Linear(4 * 4 * 32, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.maxpool1(self.relu1(self.cnn1(x)))
        out = self.maxpool2(self.relu2(self.cnn2(out)))
        out = self.linear1(out.view(x.size(0), -1))
        return out
    
#2.nn.function  作用同上

class CNN(nn.Module):
    
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.cnn1_weight = nn.Parameter(torch.rand(16, 1, 5, 5))
        self.bias1_weight = nn.Parameter(torch.rand(16))
        
        self.cnn2_weight = nn.Parameter(torch.rand(32, 16, 5, 5))
        self.bias2_weight = nn.Parameter(torch.rand(32))
        
        self.linear1_weight = nn.Parameter(torch.rand(4 * 4 * 32, 10))
        self.bias3_weight = nn.Parameter(torch.rand(10))
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = F.conv2d(x, self.cnn1_weight, self.bias1_weight)
        out = F.relu(out)
        out = F.max_pool2d(out)
        
        out = F.conv2d(x, self.cnn2_weight, self.bias2_weight)
        out = F.relu(out)
        out = F.max_pool2d(out)
        
        out = F.linear(x, self.linear1_weight, self.bias3_weight)
        return out
    
    

In [28]:
#使用nn.Xxx方式定义dropout，在调用model.eval()之后，
#model中所有的dropout layer都关闭，但以nn.function.dropout方式定义dropout，
#在调用model.eval()之后并不能关闭dropout。

#区别
#1.在定义函数层（继承nn.Module）时，init函数中应该用torch.nn,
#  例如torch.nn.ReLU，torch.nn.Dropout2d,而forward中应该用torch.nn.functionl,
#   例如torch.nn.functional.relu,不过请注意，init里面定义的是标准的网络层。
#   只有torch.nn定义的才会进行训练。torch.nn.functional定义的需要自己手动设置参数。
#    所以通常，激活函数或者卷积之类的都用torch.nn定义。
#2.torch.nn是类，必须要先在init中实例化，然后在forward中使用，
#   而torch.nn.functional可以直接在forward中使用。

import torch
import torch.nn as nn
import torch.nn.functional as F

class Model1(nn.Module):
    
    def __init__(self):
        super(Model1, self).__init__()
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        return self.dropout(x)
    
    
class Model2(nn.Module):

    def __init__(self):
        super(Model2, self).__init__()

    def forward(self, x):
        return F.dropout(x)

#help dropout
class Model3(nn.Module):

    def __init__(self):
        super(Model3, self).__init__()

    def forward(self, x):
        return F.dropout(x, training=self.training)


m1 = Model1()
m2 = Model2()
inputs = torch.rand(10)
print(m1(inputs))
print(m2(inputs))
print(20 * '-' + "eval model:" + 20 * '-' + '\r\n')
m1.eval()
m2.eval()
print(m1(inputs))
print(m2(inputs))

tensor([0.6745, 0.0000, 0.0000, 0.0000, 0.0000, 0.5155, 0.6657, 0.0000, 1.5495,
        0.0000])
tensor([0.0000, 0.4281, 0.0000, 0.0000, 0.0000, 0.0000, 0.6657, 1.7418, 0.0000,
        1.6574])
--------------------eval model:--------------------

tensor([0.3373, 0.2141, 0.8412, 0.0870, 0.2735, 0.2577, 0.3328, 0.8709, 0.7747,
        0.8287])
tensor([0.0000, 0.4281, 0.0000, 0.0000, 0.5470, 0.0000, 0.6657, 0.0000, 1.5495,
        0.0000])


In [1]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

NameError: name 'transforms' is not defined

In [4]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

NameError: name 'transforms' is not defined

In [6]:

import torch
 
def trigger_error():
    print("trigger_error")
    
    # Tensors.
    a = torch.tensor([1.0], requires_grad=True)
#error    b = torch.zeros((2), requires_grad=True)
    b = torch.zeros((2),requires_grad=False) #2
#error # CopySlices.
    b[0] = a*2  #2
    b[1] = a*3  #2
 #如果我们只有确定个数的元素需要组成一个新的tensor，
 #可以使用torch.cat()函数
    # 1 b = torch.cat((a*2,a*3))
    
    print("b.grad_fn = {}. ".format(b.grad_fn))
 
    # Trigger an error.
    b.backward(torch.ones((2)))
 
    print("a.grad = {}. ".format( a.grad ))
 
if __name__ == "__main__":
    
    trigger_error()

trigger_error
b.grad_fn = <CopySlices object at 0x7f70390843c8>. 
a.grad = tensor([5.]). 


In [8]:
seq = 'hello'
for i,key in enumerate(seq):
    print('(1):','seq[%d]=%s'%(i,key))
seq2 = ['a','b','c','d']
for i,key in enumerate(seq2):
    print('(2):','seq2[%d]=%s'%(i,key))

(1): seq[0]=h
(1): seq[1]=e
(1): seq[2]=l
(1): seq[3]=l
(1): seq[4]=o
(2): seq2[0]=a
(2): seq2[1]=b
(2): seq2[2]=c
(2): seq2[3]=d


In [2]:
#手动实现反向传播


# Code in file tensor/two_layer_net_tensor.py
import torch
 
device = torch.device('cpu')
# device = torch.device('cuda') # Uncomment this to run on GPU
 
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10
 
# Create random input and output data
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)
 
# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device)
w2 = torch.randn(H, D_out, device=device)
 
learning_rate = 1e-6
for t in range(500):
  # Forward pass: compute predicted y
  h = x.mm(w1)  #x与w1相乘
  h_relu = h.clamp(min=0)  #设置h的input上下线，此处是下限为0
  y_pred = h_relu.mm(w2)   #给model预测y赋值为激活函数h乘上一个w2参数
 
  # Compute and print loss; loss is a scalar, and is stored in a PyTorch Tensor
  # of shape (); we can get its value as a Python number with loss.item().
  loss = (y_pred - y).pow(2).sum()
  print(t, loss.item())
 
  # Backprop to compute gradients of w1 and w2 with respect to loss
  grad_y_pred = 2.0 * (y_pred - y)
  grad_w2 = h_relu.t().mm(grad_y_pred)
  grad_h_relu = grad_y_pred.mm(w2.t())
  grad_h = grad_h_relu.clone()
  grad_h[h < 0] = 0
  grad_w1 = x.t().mm(grad_h)
 
  # Update weights using gradient descent
  w1 -= learning_rate * grad_w1
  w2 -= learning_rate * grad_w2

print(w1,w2)

0 27761836.0
1 23844970.0
2 23342338.0
3 23030470.0
4 20974156.0
5 16869134.0
6 11884099.0
7 7553445.5
8 4546211.0
9 2730724.0
10 1707507.25
11 1138280.5
12 813087.625
13 617230.75
14 490821.34375
15 403333.65625
16 338967.125
17 289158.71875
18 249203.03125
19 216410.703125
20 189001.71875
21 165815.34375
22 146067.3125
23 129115.4375
24 114483.3671875
25 101796.3984375
26 90760.640625
27 81119.7265625
28 72667.53125
29 65236.1484375
30 58682.58984375
31 52884.92578125
32 47744.80859375
33 43178.86328125
34 39112.578125
35 35483.2265625
36 32237.24609375
37 29329.728515625
38 26719.185546875
39 24371.0546875
40 22256.21875
41 20346.951171875
42 18622.3203125
43 17061.654296875
44 15647.3779296875
45 14364.431640625
46 13198.7490234375
47 12138.53515625
48 11173.1494140625
49 10293.0400390625
50 9489.66796875
51 8756.0458984375
52 8084.66259765625
53 7469.7978515625
54 6906.62255859375
55 6390.08642578125
56 5916.1806640625
57 5480.4501953125
58 5079.76953125
59 4711.16015625
60 4371.6

427 0.00012474517279770225
428 0.00012221848010085523
429 0.0001200796032208018
430 0.00011778970656450838
431 0.00011541908315848559
432 0.0001137144718086347
433 0.00011132928193546832
434 0.0001090987934730947
435 0.00010738719720393419
436 0.00010553192987572402
437 0.00010336026753066108
438 0.00010164672130485997
439 9.96988092083484e-05
440 9.810032497625798e-05
441 9.609281551092863e-05
442 9.428361954633147e-05
443 9.298945951741189e-05
444 9.159750334220007e-05
445 9.016311378218234e-05
446 8.88054637471214e-05
447 8.738310862099752e-05
448 8.609439100837335e-05
449 8.481464465148747e-05
450 8.328721742145717e-05
451 8.184666512534022e-05
452 8.053946658037603e-05
453 7.900767377577722e-05
454 7.771814853185788e-05
455 7.645118603250012e-05
456 7.535493932664394e-05
457 7.40291943657212e-05
458 7.292174268513918e-05
459 7.189624739112332e-05
460 7.092976738931611e-05
461 7.010394620010629e-05
462 6.914208643138409e-05
463 6.803491851314902e-05
464 6.693548493785784e-05
465 6.

In [3]:
#autograd 自动反向传播

# Code in file autograd/two_layer_net_autograd.py
import torch
 
device = torch.device('cpu')
# device = torch.device('cuda') # Uncomment this to run on GPU
 
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10
 
# Create random Tensors to hold input and outputs
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)
 
# Create random Tensors for weights; setting requires_grad=True means that we
# want to compute gradients for these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)
 
learning_rate = 1e-6
for t in range(500):
  # Forward pass: compute predicted y using operations on Tensors. Since w1 and
  # w2 have requires_grad=True, operations involving these Tensors will cause
  # PyTorch to build a computational graph, allowing automatic computation of
  # gradients. Since we are no longer implementing the backward pass by hand we
  # don't need to keep references to intermediate values.
  y_pred = x.mm(w1).clamp(min=0).mm(w2)
  
  # Compute and print loss. Loss is a Tensor of shape (), and loss.item()
  # is a Python number giving its value.
  loss = (y_pred - y).pow(2).sum()
  print(t, loss.item())
 
  # Use autograd to compute the backward pass. This call will compute the
  # gradient of loss with respect to all Tensors with requires_grad=True.
  # After this call w1.grad and w2.grad will be Tensors holding the gradient
  # of the loss with respect to w1 and w2 respectively.
  loss.backward()
 
  # Update weights using gradient descent. For this step we just want to mutate
  # the values of w1 and w2 in-place; we don't want to build up a computational
  # graph for the update steps, so we use the torch.no_grad() context manager
  # to prevent PyTorch from building a computational graph for the updates
  with torch.no_grad():
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad
 
    # Manually zero the gradients after running the backward pass
    w1.grad.zero_()
    w2.grad.zero_()

0 31109852.0
1 28812538.0
2 27308410.0
3 23786640.0
4 18011938.0
5 11969597.0
6 7274300.0
7 4358498.0
8 2720972.5
9 1834768.125
10 1338406.375
11 1041326.0
12 848076.375
13 711151.0
14 607483.875
15 525486.0
16 458376.34375
17 402303.21875
18 354816.28125
19 314211.6875
20 279245.5625
21 248934.296875
22 222527.234375
23 199440.953125
24 179189.703125
25 161361.765625
26 145622.828125
27 131691.46875
28 119319.796875
29 108304.8671875
30 98481.546875
31 89696.703125
32 81819.1171875
33 74747.1875
34 68383.9765625
35 62654.96875
36 57489.7890625
37 52825.546875
38 48599.28125
39 44764.14453125
40 41277.92578125
41 38104.8515625
42 35215.51953125
43 32580.65625
44 30174.0703125
45 27971.841796875
46 25955.595703125
47 24106.76171875
48 22409.47265625
49 20849.88671875
50 19414.10546875
51 18091.173828125
52 16871.537109375
53 15746.3203125
54 14707.5400390625
55 13747.0380859375
56 12857.6796875
57 12033.802734375
58 11270.64453125
59 10563.0703125
60 9906.39453125
61 9296.681640625
62 8

389 0.012560858391225338
390 0.012107543647289276
391 0.011672843247652054
392 0.011257714591920376
393 0.010859287343919277
394 0.01046798750758171
395 0.010086710564792156
396 0.009730379097163677
397 0.009382086805999279
398 0.009052202105522156
399 0.008726879023015499
400 0.008419369347393513
401 0.008121725171804428
402 0.00784169789403677
403 0.0075671738013625145
404 0.0073023405857384205
405 0.007038268260657787
406 0.006795134861022234
407 0.006557194050401449
408 0.006331526208668947
409 0.006111170630902052
410 0.005903318058699369
411 0.005698333960026503
412 0.005499598570168018
413 0.005313810892403126
414 0.005127099342644215
415 0.0049514081329107285
416 0.004779623821377754
417 0.004618410486727953
418 0.004463473334908485
419 0.004311115015298128
420 0.004161748103797436
421 0.004017828963696957
422 0.003886522725224495
423 0.003758043982088566
424 0.0036321745719760656
425 0.0035112760961055756
426 0.0033951853401958942
427 0.003282967722043395
428 0.003176239551976

In [5]:
#    在PyTorch中我们可以通过定义torch.autograd.Function子类
#和实现forward和backward函数来定义我们自己的autograd操作。
#然后我们可以通过创建实例来使用新的autograd操作，并像函数一样调用，
#传播Tensor。

# Code in file autograd/two_layer_net_custom_function.py
import torch
 
class MyReLU(torch.autograd.Function):
  """
  We can implement our own custom autograd Functions by subclassing
  torch.autograd.Function and implementing the forward and backward passes
  which operate on Tensors.
  """
  @staticmethod
  def forward(ctx, x):
    """
    In the forward pass we receive a context object and a Tensor containing the
    input; we must return a Tensor containing the output, and we can use the
    context object to cache objects for use in the backward pass.
    """
    ctx.save_for_backward(x)
    return x.clamp(min=0)
 
  def backward(ctx, grad_output):
    """
    In the backward pass we receive the context object and a Tensor containing
    the gradient of the loss with respect to the output produced during the
    forward pass. We can retrieve cached data from the context object, and must
    compute and return the gradient of the loss with respect to the input to the
    forward function.
    """
    x, = ctx.saved_tensors
    grad_x = grad_output.clone()
    grad_x[x < 0] = 0
    return grad_x
device = torch.device('cpu')
# device = torch.device('cuda') # Uncomment this to run on GPU
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10
# Create random Tensors to hold input and output
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)
# Create random Tensors for weights.
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)
learning_rate = 1e-6
for t in range(500):
  # Forward pass: compute predicted y using operations on Tensors; we call our
  # custom ReLU implementation using the MyReLU.apply function
  y_pred = MyReLU.apply(x.mm(w1)).mm(w2)
 
  # Compute and print loss
  loss = (y_pred - y).pow(2).sum()
  print(t, loss.item())
  # Use autograd to compute the backward pass.
  loss.backward()
  with torch.no_grad():
    # Update weights using gradient descent
    w1 -= learning_rate * w1.grad
    w2 -= learning_rate * w2.grad
    # Manually zero the gradients after running the backward pass
    w1.grad.zero_()
    w2.grad.zero_()

0 48698772.0
1 47006088.0
2 41726524.0
3 28617540.0
4 14930096.0
5 6872810.5
6 3460589.0
7 2119709.75
8 1517763.5
9 1182610.125
10 958880.875
11 793490.625
12 664617.5625
13 561360.6875
14 477524.90625
15 408757.625
16 351852.21875
17 304433.78125
18 264612.875
19 230964.875
20 202383.6875
21 177960.171875
22 156992.953125
23 138937.4375
24 123292.234375
25 109661.640625
26 97776.9453125
27 87376.6875
28 78259.3046875
29 70237.640625
30 63156.0078125
31 56895.859375
32 51341.32421875
33 46402.29296875
34 42004.10546875
35 38084.65625
36 34581.56640625
37 31441.37890625
38 28623.4765625
39 26090.630859375
40 23809.748046875
41 21753.87890625
42 19897.265625
43 18219.23046875
44 16700.041015625
45 15322.5546875
46 14072.755859375
47 12937.005859375
48 11903.62109375
49 10962.3818359375
50 10104.24609375
51 9321.177734375
52 8605.55859375
53 7951.119140625
54 7351.8837890625
55 6802.44091796875
56 6298.55810546875
57 5835.80078125
58 5410.72509765625
59 5020.57568359375
60 4661.6440429687

432 0.00010988646681653336
433 0.00010802468023030087
434 0.00010561008093645796
435 0.00010360265150666237
436 0.00010160137026105076
437 9.970449900720268e-05
438 9.780786058399826e-05
439 9.641629003454e-05
440 9.463880996918306e-05
441 9.286097338190302e-05
442 9.113542182603851e-05
443 8.942994463723153e-05
444 8.75299156177789e-05
445 8.58327402966097e-05
446 8.454686758341268e-05
447 8.296868327306584e-05
448 8.15375242382288e-05
449 8.002867252798751e-05
450 7.880920020397753e-05
451 7.745487528154626e-05
452 7.618948438903317e-05
453 7.469373667845502e-05
454 7.389457459794357e-05
455 7.286064646905288e-05
456 7.136417116271332e-05
457 7.008256216067821e-05
458 6.900566950207576e-05
459 6.797299283789471e-05
460 6.70891095069237e-05
461 6.602618668694049e-05
462 6.505173951154575e-05
463 6.381747516570613e-05
464 6.271243910305202e-05
465 6.189177656779066e-05
466 6.082378968130797e-05
467 5.989955388940871e-05
468 5.863378100912087e-05
469 5.79707084398251e-05
470 5.728582254

In [11]:
#    在PyTorch中，nn包具有同样的作用。 nn包定义了一组模块，
#    它们大致相当于神经网络层。模块接收输入Tensor并计算输出Tensor，
#    但也可以保持内部状态，例如包含可学习参数的Tensor。
#    nn包还定义了一组在训练神经网络时常用的有用的损失函数。


# Code in file nn/two_layer_net_nn.py
import torch

 
device = torch.device('cpu')
# device = torch.device('cuda') # Uncomment this to run on GPU
 
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10
 
# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)
 
# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# After constructing the model we use the .to() method to move it to the
# desired device.
model = torch.nn.Sequential(
          torch.nn.Linear(D_in, H),
          torch.nn.ReLU(),
          torch.nn.Linear(H, D_out),
        ).to(device)
 
# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(size_average=False)
 
learning_rate = 1e-4
for t in range(500):
  # Forward pass: compute predicted y by passing x to the model. Module objects
  # override the __call__ operator so you can call them like functions. When
  # doing so you pass a Tensor of input data to the Module and it produces
  # a Tensor of output data.
  y_pred = model(x)
 
  # Compute and print loss. We pass Tensors containing the predicted and true
  # values of y, and the loss function returns a Tensor containing the loss.
  loss = loss_fn(y_pred, y)
  print(t, loss.item())

  
  # Zero the gradients before running the backward pass.
  model.zero_grad()
 
  # Backward pass: compute gradient of the loss with respect to all the learnable
  # parameters of the model. Internally, the parameters of each Module are stored
  # in Tensors with requires_grad=True, so this call will compute gradients for
  # all learnable parameters in the model.
  loss.backward()
 
  # Update the weights using gradient descent. Each parameter is a Tensor, so
  # we can access its data and gradients like we did before.
  with torch.no_grad():
    for param in model.parameters():
      param.data -= learning_rate * param.grad



0 733.460205078125
1 675.2667846679688
2 624.9657592773438
3 581.22998046875
4 542.4168701171875
5 507.5355224609375
6 476.0512390136719
7 447.3106689453125
8 420.8769836425781
9 396.2803039550781
10 373.37091064453125
11 351.8489990234375
12 331.7290954589844
13 312.8040466308594
14 294.7820129394531
15 277.69915771484375
16 261.4081115722656
17 245.91004943847656
18 231.2219696044922
19 217.28248596191406
20 204.03433227539062
21 191.5157012939453
22 179.6929473876953
23 168.5435791015625
24 158.01904296875
25 148.1019287109375
26 138.7822265625
27 130.0263671875
28 121.8121566772461
29 114.09151458740234
30 106.84748840332031
31 100.05162811279297
32 93.70541381835938
33 87.77469635009766
34 82.24079895019531
35 77.05770874023438
36 72.19026184082031
37 67.63948059082031
38 63.38191604614258
39 59.371551513671875
40 55.623329162597656
41 52.12847137451172
42 48.869815826416016
43 45.83354568481445
44 42.99934005737305
45 40.34906005859375
46 37.87310028076172
47 35.55834197998047
48

361 0.0004657409735955298
362 0.00045460023102350533
363 0.0004437332972884178
364 0.00043312559137120843
365 0.00042279099579900503
366 0.0004126919084228575
367 0.00040285199065692723
368 0.00039325153920799494
369 0.00038390106055885553
370 0.0003747536102309823
371 0.00036583386827260256
372 0.0003571310662664473
373 0.00034864709596149623
374 0.0003403625451028347
375 0.00033227764652110636
376 0.0003243916726205498
377 0.0003166954847984016
378 0.0003091850667260587
379 0.00030186300864443183
380 0.00029471423476934433
381 0.0002877400547731668
382 0.000280928798019886
383 0.00027428477187640965
384 0.0002678038727026433
385 0.00026147637981921434
386 0.0002553029917180538
387 0.0002492734638508409
388 0.00024339626543223858
389 0.00023765477817505598
390 0.0002320536586921662
391 0.00022658455418422818
392 0.0002212454710388556
393 0.0002160400035791099
394 0.0002109564666170627
395 0.00020599467097781599
396 0.00020115316146984696
397 0.00019642282859422266
398 0.00019180693197

In [12]:
#   PyTorch中的optim包抽象出优化算法的思想，并提供常用优化算法的实现。
#   其采用很多高级方法来获取更优的梯度方向并更新梯度。

#    在这个例子中，我们将像以前一样使用nn包定义我们的模型，
#    但我们将使用optim包提供的Adam算法优化模型：

# Code in file nn/two_layer_net_optim.py
import torch
 
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10
 
# Create random Tensors to hold inputs and outputs.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
 
# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
          torch.nn.Linear(D_in, H),
          torch.nn.ReLU(),
          torch.nn.Linear(H, D_out),
        )
loss_fn = torch.nn.MSELoss(size_average=False)
 
# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
  # Forward pass: compute predicted y by passing x to the model.
  y_pred = model(x)
 
  # Compute and print loss.
  loss = loss_fn(y_pred, y)
  print(t, loss.item())
  
  # Before the backward pass, use the optimizer object to zero all of the
  # gradients for the Tensors it will update (which are the learnable weights
  # of the model)
  optimizer.zero_grad()
 
  # Backward pass: compute gradient of the loss with respect to model parameters
  loss.backward()
 
  # Calling the step function on an Optimizer makes an update to its parameters
  optimizer.step()

0 654.140625
1 637.6131591796875
2 621.5366821289062
3 605.8927001953125
4 590.781494140625
5 576.076171875
6 561.779296875
7 547.9662475585938
8 534.6185302734375
9 521.6929931640625
10 509.1822204589844
11 497.0024719238281
12 485.204833984375
13 473.72125244140625
14 462.6139221191406
15 451.83782958984375
16 441.3051452636719
17 431.09210205078125
18 421.1272888183594
19 411.454833984375
20 402.05853271484375
21 392.8824768066406
22 383.9479064941406
23 375.2609558105469
24 366.7999267578125
25 358.5826416015625
26 350.5694580078125
27 342.7445983886719
28 335.10382080078125
29 327.6232604980469
30 320.33929443359375
31 313.2378234863281
32 306.2790832519531
33 299.4718933105469
34 292.82830810546875
35 286.3410339355469
36 279.992919921875
37 273.786865234375
38 267.6823425292969
39 261.6813049316406
40 255.81317138671875
41 250.0522003173828
42 244.39830017089844
43 238.8625030517578
44 233.4343719482422
45 228.09872436523438
46 222.87620544433594
47 217.74191284179688
48 212.702

376 3.8770052924519405e-05
377 3.637297049863264e-05
378 3.41178456437774e-05
379 3.200789069524035e-05
380 3.0021308702998795e-05
381 2.8158878194517456e-05
382 2.6408746634842828e-05
383 2.476986264809966e-05
384 2.3228709324030206e-05
385 2.1782507246825844e-05
386 2.0427227354957722e-05
387 1.9153581888531335e-05
388 1.7957454474526457e-05
389 1.6836327631608583e-05
390 1.5783864000695758e-05
391 1.4795539755141363e-05
392 1.3868058886146173e-05
393 1.3000928447581828e-05
394 1.2184388651803602e-05
395 1.1420086593716405e-05
396 1.0701645805966109e-05
397 1.0028332326328382e-05
398 9.396957466378808e-06
399 8.804320714261848e-06
400 8.24918424768839e-06
401 7.728336640866473e-06
402 7.240387276397087e-06
403 6.782225227652816e-06
404 6.353176559059648e-06
405 5.949931164650479e-06
406 5.571591373154661e-06
407 5.2187397159286775e-06
408 4.886891929345438e-06
409 4.574994363792939e-06
410 4.283136149751954e-06
411 4.010646989627276e-06
412 3.7549618809862295e-06
413 3.51428457179281

In [13]:
#PyTorch: Custom nn Modules
    #有时需要指定比现有模块序列更复杂的模型；对于这些情况，
    #可以通过子类化nn.Module定义自己的模块，并定义一个接收输入Tensor的forward，
    #并使用其他模块或Tensor上的其他autograd操作生成输出Tensor。

   #在这个例子中，我们将以自定义Module子类的形式实现2层网络：


# Code in file nn/two_layer_net_module.py
import torch
 
class TwoLayerNet(torch.nn.Module):
  def __init__(self, D_in, H, D_out):
    """
    In the constructor we instantiate two nn.Linear modules and assign them as
    member variables.
    """
    super(TwoLayerNet, self).__init__()
    self.linear1 = torch.nn.Linear(D_in, H)
    self.linear2 = torch.nn.Linear(H, D_out)
 
  def forward(self, x):
    """
    In the forward function we accept a Tensor of input data and we must return
    a Tensor of output data. We can use Modules defined in the constructor as
    well as arbitrary (differentiable) operations on Tensors.
    """
    h_relu = self.linear1(x).clamp(min=0)
    y_pred = self.linear2(h_relu)
    return y_pred
 
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10
 
# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
 
# Construct our model by instantiating the class defined above.
model = TwoLayerNet(D_in, H, D_out)
 
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
loss_fn = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
  # Forward pass: Compute predicted y by passing x to the model
  y_pred = model(x)
 
  # Compute and print loss
  loss = loss_fn(y_pred, y)
  print(t, loss.item())
 
  # Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

0 706.7421264648438
1 656.128173828125
2 612.373779296875
3 573.9771728515625
4 539.90625
5 509.0140686035156
6 481.1817626953125
7 455.56658935546875
8 432.02880859375
9 410.1608581542969
10 389.59515380859375
11 370.27874755859375
12 351.9056396484375
13 334.4429626464844
14 317.74468994140625
15 301.78717041015625
16 286.48565673828125
17 271.8874816894531
18 257.8059387207031
19 244.3289794921875
20 231.46481323242188
21 219.14947509765625
22 207.34396362304688
23 196.06674194335938
24 185.26617431640625
25 174.8857879638672
26 164.99172973632812
27 155.5475616455078
28 146.5657958984375
29 138.02297973632812
30 129.89414978027344
31 122.2136001586914
32 114.91441345214844
33 108.01841735839844
34 101.4728012084961
35 95.29704284667969
36 89.47154998779297
37 83.98629760742188
38 78.83023071289062
39 73.99015045166016
40 69.43306732177734
41 65.15087127685547
42 61.13054656982422
43 57.34489059448242
44 53.792930603027344
45 50.470481872558594
46 47.35105514526367
47 44.42506027221

435 7.3355140557396226e-06
436 7.119142537703738e-06
437 6.91042350808857e-06
438 6.707799002469983e-06
439 6.510881576105021e-06
440 6.319968633761164e-06
441 6.136096089903731e-06
442 5.9554859035415575e-06
443 5.782007974630687e-06
444 5.612937002297258e-06
445 5.449127456813585e-06
446 5.289417003950803e-06
447 5.136986601428362e-06
448 4.987489319319138e-06
449 4.842167982133105e-06
450 4.70160057375324e-06
451 4.564796199701959e-06
452 4.4323555812297855e-06
453 4.304361027607229e-06
454 4.179496045253472e-06
455 4.0578261177870445e-06
456 3.939996986446204e-06
457 3.827463388006436e-06
458 3.715929324243916e-06
459 3.60977060154255e-06
460 3.5056204978900496e-06
461 3.403970595172723e-06
462 3.305285190435825e-06
463 3.210499471606454e-06
464 3.118645508948248e-06
465 3.028424998774426e-06
466 2.941754701168975e-06
467 2.8571434995683376e-06
468 2.7754861093853833e-06
469 2.695406692510005e-06
470 2.6181971861660713e-06
471 2.543077243899461e-06
472 2.470536401233403e-06
473 2.4

In [14]:
#PyTorch: Control Flow + Weight Sharing

    #作为动态图和权重共享的一个例子，我们实现了一个非常奇怪的模型：
    #一个完全连接的ReLU网络，在每个前向传递中选择1到4之间的随机数并使用
    #那么多隐藏层，重复使用相同的权重多次 计算最里面的隐藏层。
    
# Code in file nn/dynamic_net.py
import random
import torch
 
class DynamicNet(torch.nn.Module):
  def __init__(self, D_in, H, D_out):
    """
    In the constructor we construct three nn.Linear instances that we will use
    in the forward pass.
    """
    super(DynamicNet, self).__init__()
    self.input_linear = torch.nn.Linear(D_in, H)
    self.middle_linear = torch.nn.Linear(H, H)
    self.output_linear = torch.nn.Linear(H, D_out)
 
  def forward(self, x):
    """
    For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
    and reuse the middle_linear Module that many times to compute hidden layer
    representations.
 
    Since each forward pass builds a dynamic computation graph, we can use normal
    Python control-flow operators like loops or conditional statements when
    defining the forward pass of the model.
 
    Here we also see that it is perfectly safe to reuse the same Module many
    times when defining a computational graph. This is a big improvement from Lua
    Torch, where each Module could be used only once.
    """
    h_relu = self.input_linear(x).clamp(min=0)
    for _ in range(random.randint(0, 3)):
      h_relu = self.middle_linear(h_relu).clamp(min=0)
    y_pred = self.output_linear(h_relu)
    return y_pred
 
 
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10
 
# Create random Tensors to hold inputs and outputs.
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)
 
# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)
 
# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
  # Forward pass: Compute predicted y by passing x to the model
  y_pred = model(x)
 
  # Compute and print loss
  loss = criterion(y_pred, y)
  print(t, loss.item())
 
  # Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

0 713.18603515625
1 740.5426635742188
2 698.7960205078125
3 696.6361694335938
4 590.2883911132812
5 691.9925537109375
6 469.714599609375
7 689.7020874023438
8 359.3372802734375
9 306.2121276855469
10 683.88232421875
11 657.9124145507812
12 647.6469116210938
13 626.8375854492188
14 141.60354614257812
15 567.3827514648438
16 667.035400390625
17 92.56871032714844
18 460.34869384765625
19 76.26693725585938
20 385.497802734375
21 343.85101318359375
22 296.329833984375
23 107.47029113769531
24 207.9883575439453
25 499.7738037109375
26 142.9925994873047
27 141.30697631835938
28 128.86427307128906
29 353.2174377441406
30 487.99383544921875
31 270.0692138671875
32 374.9713439941406
33 70.34030151367188
34 61.364845275878906
35 270.6654357910156
36 39.874290466308594
37 36.63257598876953
38 300.88623046875
39 28.848243713378906
40 323.704345703125
41 119.5709457397461
42 290.1915283203125
43 127.08360290527344
44 94.26490020751953
45 95.5934829711914
46 218.3485870361328
47 194.7862548828125
48 

369 0.48392876982688904
370 0.2869832217693329
371 0.13312095403671265
372 3.0087881088256836
373 0.4114905893802643
374 0.4114648699760437
375 3.425373077392578
376 1.395589828491211
377 1.0489051342010498
378 1.4234964847564697
379 1.4486114978790283
380 3.778827667236328
381 0.127614364027977
382 3.151441812515259
383 2.7122342586517334
384 0.3462696373462677
385 0.1545257419347763
386 1.2512799501419067
387 0.9746799468994141
388 1.0821336507797241
389 1.4734206199645996
390 1.253603219985962
391 0.6555903553962708
392 0.1723790466785431
393 3.290722370147705
394 0.8648631572723389
395 1.615875005722046
396 2.220625877380371
397 5.768775939941406
398 1.457713007926941
399 0.2519408166408539
400 0.6289272308349609
401 1.1503187417984009
402 5.990814685821533
403 0.34729140996932983
404 0.9311034679412842
405 0.5965750813484192
406 8.903633117675781
407 0.12437552958726883
408 1.676486849784851
409 3.317774772644043
410 1.8032708168029785
411 1.247710943222046
412 4.4474005699157715


In [14]:
#https://www.jianshu.com/p/3963b2cdd771

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



class Net(nn.Module):
    """docstring for Net"""
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)        
        self.fc3 = nn.Linear(84, 10)

    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1,16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()


criterion = nn.CrossEntropyLoss()
#声明使用交叉熵函数作为代价函数
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)
#声明使用学习率0.001的SGD优化器

for epoch in range(2):
    running_loss = 0
    for i,data in enumerate(trainloader,0):
        inputs,labels = data
        inputs,labels = Variable(inputs),Variable(labels)
        #获得数据并将其放在GPU上
        optimizer.zero_grad()
        #初始化梯度

        outputs = net(inputs)
        #前馈
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        #反馈计算梯度并更新权值

        running_loss += loss.item()    #error:    loss.data[0]
        if i % 200 == 0:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0
            #打印平均代价函数值
print('Finished Training')


corret,total = 0,0
for images,labels in testloader:
    images = images
    labels = labels
    outputs = net(Variable(images))
    _,predicted = torch.max(outputs.data,1)
    total += labels.size(0)
    corret += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * corret / total))


Files already downloaded and verified
Files already downloaded and verified
[1,     1] loss: 0.001
[1,   201] loss: 0.230
[1,   401] loss: 0.230
[1,   601] loss: 0.230
[1,   801] loss: 0.228
[1,  1001] loss: 0.225
[1,  1201] loss: 0.218
[1,  1401] loss: 0.209
[1,  1601] loss: 0.204
[1,  1801] loss: 0.203
[1,  2001] loss: 0.195
[1,  2201] loss: 0.193
[1,  2401] loss: 0.195
[1,  2601] loss: 0.194
[1,  2801] loss: 0.189
[1,  3001] loss: 0.182
[1,  3201] loss: 0.186
[1,  3401] loss: 0.181
[1,  3601] loss: 0.174
[1,  3801] loss: 0.173
[1,  4001] loss: 0.177
[1,  4201] loss: 0.174
[1,  4401] loss: 0.174
[1,  4601] loss: 0.166
[1,  4801] loss: 0.162
[1,  5001] loss: 0.167
[1,  5201] loss: 0.162
[1,  5401] loss: 0.167
[1,  5601] loss: 0.167
[1,  5801] loss: 0.159
[1,  6001] loss: 0.157
[1,  6201] loss: 0.159
[1,  6401] loss: 0.155
[1,  6601] loss: 0.163
[1,  6801] loss: 0.161
[1,  7001] loss: 0.154
[1,  7201] loss: 0.159
[1,  7401] loss: 0.149
[1,  7601] loss: 0.160
[1,  7801] loss: 0.157
[1, 

In [1]:
import torch

In [3]:
import torch as t
x = t.rand(5,3)
y = t.rand(5,3)
if t.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    print(x+y)

tensor([[0.2345, 0.6745, 1.1587],
        [1.0616, 1.4417, 1.3022],
        [1.2783, 0.1463, 0.4098],
        [0.9364, 0.8576, 0.9126],
        [0.6517, 1.3815, 0.9280]], device='cuda:0')


In [4]:
import torch
print(torch.cuda.is_available())

True
